<a href="https://colab.research.google.com/github/Mingjie-Shen/Deeep-Learning-Homework-and-Practice/blob/main/HW2_CIFAR10_ConvNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

Data Augmentation Step

Transforming the 3*32*32 array to a PIL image format -> Perform Data Transformations 
1. Randomly crops 0.5 to 1 percentage of out of the input 3*32*32 image and resizes to 3*24*24 image 
2. Randomly flip the image horizontally with a probability of 0.5
3. Randomly brighten the image with a factor selected uniformly from [max(0, 1 - brightness), 1 + brightness].
4. Randomly contrast the image with a factor selected uniformly from [max(0, 1 - contrast), 1 + contrast].
-> Transform PIL image back to Tensor format


In [2]:
#Defining the data augmentation transformations
transformations = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      #transforms.ColorJitter(brightness=0.3, contrast=0, saturation=0, hue=0),                              
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train = torchvision.datasets.CIFAR10(root='./CIFARdata', train=True,
                                        download=True, transform=transformations) #data augmentation transformations
data_train = torch.utils.data.DataLoader(train, batch_size=100, #Batch size = 100
                                          shuffle=True, num_workers=0)
#Transforming data and normalizing for the test set
transform_testset = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test = torchvision.datasets.CIFAR10(root='./CIFARdata', train=False,
                                       download=True, transform=transform_testset)
data_test = torch.utils.data.DataLoader(test, batch_size=100,
                                         shuffle=False, num_workers=0)

Extracting ./CIFARdata/cifar-10-python.tar.gz to ./CIFARdata
Files already downloaded and verified


In [8]:
##########

# Define deep convolution network model here
class CIFAR10Model(nn.Module):
  def __init__(self,num_outputs):
    super(CIFAR10Model,self).__init__()

    #the input size=(3,32,32)
    self.conv1=nn.Sequential(nn.Conv2d(3,64,kernel_size=4,padding=2),#output=(64,33,33)
                             nn.ReLU(),
                             nn.BatchNorm2d(64))
    
    #the input size=(64,33,33)
    self.conv2=nn.Sequential(nn.Conv2d(64,64,kernel_size=4,padding=2),#output=(64,34,34)
                             nn.ReLU(),
                             nn.MaxPool2d(kernel_size=2,stride=2),#output=(64,17,17)
                             nn.Dropout2d(0.2))
    
    #the input size=(64,17,17)
    self.conv3=nn.Sequential(nn.Conv2d(64,64,kernel_size=4,padding=2),#output=(64,18,18)
                             nn.ReLU(),
                             nn.BatchNorm2d(64))
    
    #the input size=(64,18,18)
    self.conv4=nn.Sequential(nn.Conv2d(64,64,kernel_size=4,padding=2),#output=(64,19,19)
                             nn.ReLU(),
                             nn.MaxPool2d(kernel_size=2,stride=2),#output=(64,9,9)
                             nn.Dropout2d(0.2))
    
    #the input size=(64,9,9)
    self.conv5=nn.Sequential(nn.Conv2d(64,64,kernel_size=4,padding=2),#output=(64,10,10)
                             nn.ReLU(),
                             nn.BatchNorm2d(64))
    
    #the input size=(64,10,10)
    self.conv6=nn.Sequential(nn.Conv2d(64,64,kernel_size=3),#output=(64,8,8)
                             nn.ReLU(),
                             nn.Dropout2d(0.2))
    
    #the input size=(64,8,8)
    self.conv7=nn.Sequential(nn.Conv2d(64,64,kernel_size=3),#output=(64,6,6)
                             nn.ReLU(),
                             nn.BatchNorm2d(64))
    
    #the input size=(64,6,6)
    self.conv8=nn.Sequential(nn.Conv2d(64,64,kernel_size=3),#output=(64,4,4)
                             nn.ReLU(),
                             nn.BatchNorm2d(64),
                             nn.Dropout2d(0.2))
    
    #the input size=(64,4,4)
    self.fc1=nn.Linear(64*4*4,500)
    self.fc2=nn.Linear(500,500)
    self.fc3=nn.Linear(500,num_outputs)

  def forward(self,x):
    x=self.conv1(x)
    x=self.conv2(x)
    x=self.conv3(x)
    x=self.conv4(x)
    x=self.conv5(x)
    x=self.conv6(x)
    x=self.conv7(x)
    x=self.conv8(x)
    x=self.fc1(x.reshape(-1,64*4*4))
    x=self.fc2(x)
    x=self.fc3(x)
    return F.log_softmax(x,dim=1)

In [ ]:
#Parameters
num_outputs = 10 #10 classes
heuristic = True #Choosing from heuristic and Monte Carlo approximation

model = CIFAR10Model(num_outputs)
#model.load_state_dict(torch.load('params_cifar10_dcnn_LR001.ckpt')) #To load a saved model 
print ('\nModel Architecture is:\n', model)
model.cuda() #Sending the model to the GPU
batch_size = 100  #Batch size
loss_func = nn.CrossEntropyLoss() #Cross entropy loss function
if heuristic:
    model.train()
LR = 0.001 #Learning rate 
train_accuracy = []
'''
Train Mode
Create the batch -> Zero the gradients -> Forward Propagation -> Calculating the loss 
-> Backpropagation -> Optimizer updating the parameters -> Prediction 
'''
start_time = time.time()
for epoch in range(250):  # loop over the dataset multiple times
    #Defining the learning rate based on the no. of epochs
    if (epoch > 50):
        LR = 0.001
    if (epoch > 100):
        LR = 0.0001
    if (epoch > 200):
        LR = 0.00001
    optimizer = optim.Adam(model.parameters(), lr=LR) #ADAM optimizer
    running_loss = 0.0
    for i, batch in enumerate(data_train, 0):
        data, target = batch
        data, target = Variable(data).cuda(), Variable(target).cuda()
        optimizer.zero_grad() #Zero the gradients at each epoch
        output = model(data)#Forward propagation
        #Negative Log Likelihood Objective function
        loss = loss_func(output, target)
        loss.backward() #Backpropagation 
        optimizer.step() #Updating the parameters using ADAM optimizer
        prediction = output.data.max(1)[1] #Label Prediction 
        accuracy = (float(prediction.eq(target.data).sum())/float(batch_size))*100.0 #Computing the training accuracy
        train_accuracy.append(accuracy)   
    accuracy_epoch = np.mean(train_accuracy)
    print('\nIn epoch ', epoch,' the accuracy of the training set =', accuracy_epoch)
end_time = time.time() 


Model Architecture is:
 CIFAR10Model(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.2, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.2, inplace=False)
  )
  (conv5): Sequential(
    (0): Conv2d(64, 64, kernel_size=(4,

In [ ]:
#torch.save(model.state_dict(), 'params_cifar10_dcnn_LR001.ckpt') #To save the trained model 
'''
Calculate accuracy of trained model on the Test Set
Create the batch ->  Forward Propagation -> Prediction 
'''
correct = 0
total = 0
test_accuracy = []

In [ ]:
if heuristic:
    MonteCarlo = 1 #Only one itertion if we are using the heuristic 
    #Comparing the accuarcy of the heuristic and Monte Carlo Method  
    model.eval()
    #Disable the droupout layer during test time
    #This scales the activation with probability of dropout p at test time 
    #So this is applying the heuristic at the test time
else:
    MonteCarlo = 100 #100 Monte Carlo iterations for comparision with heuristic method.
    
for batch in data_test:
    data, target = batch
    data, target  = Variable(data).cuda(), Variable(target).cuda()
    for k in range (MonteCarlo):
        if k ==0:
            output = model(data)  #Forward propagation     
        else:
            output+=model(data)  #Forward propagation     
    output/=MonteCarlo #Averaging the softmmax probabilities montecarlo no. of times 
    prediction = output.data.max(1)[1] #Label Prediction
    accuracy = (float(prediction.eq(target.data).sum())/float(batch_size))*100.0 #Computing the test accuracy
    test_accuracy.append(accuracy)
accuracy_test2 = np.mean(test_accuracy)
print('\nAccuracy on the test set = ', accuracy_test2)